In [1]:
from gevent import monkey as curious_george
curious_george.patch_all(thread=False, select=False)
import sys
import requests
import grequests
import pandas as pd
import numpy as np
import re
import click 
from bs4 import BeautifulSoup
import time

C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
baseurl ="https://www.goodreads.com/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
}
url_links = []
for x in range(1):
    r = requests.get(f"https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page={x}")
    soup = BeautifulSoup(r.content, 'lxml')
    book_pages = soup.find_all('tr', {'itemtype':'http://schema.org/Book'})
    for item in book_pages:
        link = item.find('a',{'itemprop':'url'})['href']         
        url_links.append(baseurl + link)
print(len(url_links))


100


In [ ]:
#url, title, author, num_reviews, num_ratings, avg_rating, num_pages, original_publish_year, series, genres, awards, places

In [31]:
#for link in url_links:
testlink = "https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird"
r = requests.get(testlink,headers=headers)
soup= BeautifulSoup(r.content,'html.parser')
desc = soup.find_all('div', class_ = 'mainContentFloat')


try:  
    title = soup.find ('h1').text.strip() #WORKS
except :
    title = np.nan
try:
    avg_ratings = soup.find('span', itemprop = "ratingValue").text.strip()
    
except:
    avg_ratings= np.nan
try:
    rating_count = soup.find('meta',itemprop="ratingCount").text.strip()
except:
    rating_count = np.nan
try:
    reviews = soup.find('meta',itemprop="reviewCount").text.strip()#WORKS
    reviews = re.sub("\D", "", reviews)
except:
    reviews=np.nan
try:
    authors = soup.find('a', class_ = "authorName").text.strip()#WOrks
except:
    authors= np.nan

try:
    pages = item.find('span', itemprop = "numberOfPages").text
    pages = re.sub("\D", "", pages)
except:
    pages = np.nan
try:
    original_publish_year = item.find('nobr',{'class':'greyText'}).text
except:
    original_publish_year  = np.nan
genre = []
for i in soup.find_all('a',{'class':"actionLinkLite bookPageGenreLink"}):

    genre.append(i.get_text()) #WORKS

series = soup.find('div', id = "bookDataBox").text
if "Series" in series: 
    series = '1'
else: 
    series = '0'

awards_list = []    
names = soup.find_all('a', class_="award")
if names is not None:
    for name in names:
            awards_list.append(name.get_text())





data = {"url":link,
        'title':title,
        'authors': authors,
        'reviews': reviews,
        'ratings': rating_count,
        'avg_rating': avg_ratings,
        'num_pages': pages,
        'original_publish_year': original_publish_year,
        'series': series,
        'genre':genre,
        'awards': awards_list

        }
print(data)



ValueError: arrays must all be same length

In [30]:
data

{'url': 'https://www.goodreads.com//book/show/153747.Moby_Dick_or_the_Whale',
 'title': 'To Kill a Mockingbird',
 'avg_ratings': '351',
 'authors': 'Harper Lee',
 'reviews': '95094',
 'ratings': '4,748,049\n  ratings',
 'avg_rating': '4.28',
 'num_pages': nan,
 'original_publish_year': nan,
 'series': '1',
 'genre': ['Classics',
  'Fiction',
  'Historical',
  'Historical Fiction',
  'Academic',
  'School',
  'Literature',
  'Young Adult',
  'Historical',
  'Novels',
  'Academic',
  'Read For School',
  'Young Adult',
  'High School'],
 'awards': ['Pulitzer Prize for Fiction (1961)',
  'Audie Award for Classic (2007)',
  'National Book Award Finalist for Fiction (1961)',
  'Alabama Author Award for Fiction (1961)']}

In [ ]:
try:
    total_ratings = soup.find('span', itemprop = "ratingValue").text.strip()
    total_ratings=re.sub("\D", "", total_ratings)
except:
    total_ratings='NOPE'
try:
    reviews = soup.find('meta',{"itemprop":"reviewCount"}).text.strip()#WORKS
    reviews = re.sub("\D", "", reviews)
except:
    reviews='NOPE'
try:
    authors = soup.find('a', class_ = "authorName").text.strip()#WOrks
except:
    authors= 'NOPE'
try:
    pages = item.find('span', itemprop = "numberOfPages").text
    pages = re.sub("\D", "", pages)
except:
    pages = 'NOPE'
try:
    year = item.find('nobr',{'class':'greyText'}).text
except:
    year = 'NOPE'



{'title': 'Moby-Dick or, the Whale',
 'total_ratings': '491,914\n  ratings',
 'authors': 'by\n\n\nHerman Melville, \n\n\nAndrew Delbanco (Introduction), \n\n\nTom Quirk (Notes), \n\n\nRockwell Kent (Illustrator)',
 'reviews': '16,621\n    reviews',
 'pages': 'NOPE',
 'year': 'NOPE'}

In [2]:
base_url = "https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once{}"


def get_pages_links(string, index):
    links = []
    for index in range(1,index):
        url = string.format(index)
        links.append(url)
    return links
    
def grequest_page(strings, index):
    reqs = (grequests.get(string) for string in strings)
    resp = grequests.imap(reqs, grequests.Pool(index))
    return resp
def request_single_page(string):
    res = requests.get(string)
    return res

def request_soup(page):
    soup = BeautifulSoup(page.content ,"html.parser")
    return soup

def close_request(request_page):
    request_page.close()
    return


In [1]:




def read_books_links(page_soup):
    Links = []
    #table = page_soup.find_all('table', class_ = 'tableList js-dataTooltip')
    #print(table)
    books = page_soup.find_all('tr')
    for book in books:
        info_book = book.find('a', class_ = 'bookTitle')
        book_link = "https://www.goodreads.com{}".format(info_book['href'])
        Links.append(book_link)    
    return Links
def get_bookslink(index):
    Links_res = []
    page = grequest_page(get_pages_links(base_url, index), index)
    for r in page: 
        soup = request_soup(r)
        Links_res = Links_res + read_books_links(soup)
    print(len(Links_res))
    with open("Links_for_each_book.txt", "w") as output:
        output.write(str(Links_res))
    return Links_res

def get_pupDate(data):  
    if data.find('nobr', class_ = "greyText") is not None:
        firstPub = data.find('nobr', class_ = "greyText").text
        firstPub = firstPub.strip()
        firstPub = firstPub[-5:]
        firstPub = firstPub.replace(")","")
        return firstPub
    else: return np.nan

def three_genres(book):
    genres = []
    names = book.find_all('a', class_="actionLinkLite bookPageGenreLink")
    if names is not None:
        for name in names:
          genres.append(name.get_text())
          genres = genres[:3]  
        return genres
    else:
        return np.nan

def get_awards(book):
    awards_list = []    
    names = book.find_all('a', class_="award")
    if names is not None:
        for name in names:
            awards_list.append(name.get_text())
        return awards_list
    else: 
        return np.nan
    
def get_places(book):
    return

def get_info_book(page_soup, link, index):
    book = page_soup.find_all('div', class_ = 'mainContentFloat')
    for data in book:
        #TITLE of the Book
        title = data.find('h1')
        if title is not None:
            title = title.text
            title = title.strip()
        else: title = np.nan
        #AUTHOR of the book
        author = data.find('a', class_ = "authorName")
        if author is not None:
            author = author.text
        else: author = np.nan
        #RATING COUNT of the book
        ratingCount = data.find('meta', itemprop = "ratingCount")
        if ratingCount is not None:
            ratingCount = ratingCount.text
            ratingCount = re.sub("\D", "", ratingCount)
        else: ratingCount = np.nan
        #REVIEW COUNT of the book
        reviewCount = data.find('meta', itemprop = "reviewCount")
        if reviewCount is not None:
            reviewCount = reviewCount.text
            reviewCount = re.sub("\D", "", reviewCount)
        else: reviewCount = np.nan
        #RATING VALUE of the book
        ratingValue = data.find('span', itemprop = "ratingValue")
        if ratingValue is not None:
            ratingValue = ratingValue.text
            ratingValue = ratingValue.strip()
        else: ratingValue = np.nan
        #NUMBER OF PAGES of the book
        numberOfPages = data.find('span', itemprop = "numberOfPages")
        if numberOfPages is not None:
            numberOfPages = numberOfPages.text
            numberOfPages = re.sub("\D", "", numberOfPages)
        else: numberOfPages = np.nan
        #YEAR OF FIRST PUBBLICATION of the book
        firstPub = get_pupDate(data)
        #CHECK IF IT IS A SERIES OR NOT
        series = data.find('div', id = "bookDataBox").text
        if "Series" in series: 
            series = '1'
        else: 
            series = '0'
        #GENRES of the book
        genreList = three_genres(data)
        #AWARDS of the book
        awards = get_awards(data)
        #PLACES of the book
 
        
        #Return Dictionary
        Book_dict = {
                "Link":link,
                "Title":title,
                "Author":author,
                "Rating Count":ratingCount,
                "Review Count":reviewCount,
                "Rating Value":ratingValue,
                "N pag":numberOfPages,
                "1st Pub":firstPub,
                "series":series,
                "Genres":genreList,
                "Awards":awards}
        #print(index,"  ",link,"\n__",title,author,ratingCount, reviewCount, ratingValue, numberOfPages,firstPub,           series, genreList, awards,"\n\n")
        return Book_dict



def create_csv(index):
    links = get_bookslink(index + 1)
    res_dict = {}
    i = 1
    page = grequest_page(links, index + 1)
    df = pd.DataFrame()
    for r, link in zip(page, links): 
        soup = request_soup(r)
        value = get_info_book(soup, link, i)
        if value is not None:
            df = df.append(value, ignore_index=True)
        i = i + 1
        print(df.tail())
    df.to_csv('./Books.csv')
    return

if __name__ == "__main__":    
    create_csv(1)
#    print("--- %s seconds ---" % (time.time() - start_time))


NameError: name 'grequest_page' is not defined

In [34]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
#mport sweetviz as sv
from matplotlib import cm
import seaborn as sns
import scipy.stats as st
import statsmodels as sm
from scipy.stats import pearsonr
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'